In [60]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

T = 1000
BATCH_SIZE = 16
# 探索torch的normal分布
e = torch.normal(mean=torch.arange(1., 11.), std=torch.arange(1, 0, -0.1))
e = torch.randn((10000, 10))
mean = torch.mean(e, dim=0)
var = torch.var(e, dim=0)
print(f'mean: {mean}, var: {var}')

mean: tensor([ 0.0039,  0.0116,  0.0173,  0.0107, -0.0036, -0.0065, -0.0083, -0.0042,
         0.0016, -0.0098]), var: tensor([1.0112, 1.0186, 1.0239, 0.9929, 1.0001, 0.9946, 0.9845, 0.9929, 1.0022,
        0.9675])


In [64]:
# 测试嵌入时间步信息
pos_embedding = nn.Embedding(T, BATCH_SIZE) 
t = torch.randint(0, T, (10, 1))
pos_embedding(t).shape

torch.Size([10, 1, 16])

In [40]:
# 切片和广播
a = torch.randint(0, 10, (2, 3))
print(a[:,1:2])
print(a[...,1:2])

T = 150
t = nn.Linear(T, 3)
time_embed = t(torch.randn(1, T))
print(time_embed.shape)
time_embed = time_embed[(..., ) + (None, ) * 2]
print(time_embed.shape)
(..., ) + (None, )*2

x = torch.zeros(1, 3, 16, 16)
x = x + time_embed
x.shape

tensor([[3],
        [0]])
tensor([[3],
        [0]])
torch.Size([1, 3])
torch.Size([1, 3, 1, 1])


torch.Size([1, 3, 16, 16])

In [59]:
t = nn.Embedding(T, 128)
time_embed = t(torch.arange(T))
time_embed.shape

torch.Size([150, 128])

In [ ]:
class SinusoidalPositionalEmbedding(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim
    
    def forward(self, t: torch.Tensor):
        half_dim = self.dim // 2
        emb = np.log(10000) / (half_dim - 1)
        emb = torch.exp(torch.arange(half_dim, dtype=torch.float32) * -emb)
        emb = t.unsqueeze(-1) * emb.unsqueeze(0)
        emb = torch.cat([torch.sin(emb), torch.cos(emb)], dim=-1)
        return emb

test = SinusoidalPositionalEmbedding(128)

In [14]:
# 探索反卷积的用法

a = torch.randn(1, 3, 256, 256)
a_conv_transpose = nn.ConvTranspose2d(3, 6, 3, stride=2, padding=1, output_padding=1)
print(a_conv_transpose(a).shape)

torch.Size([1, 6, 512, 512])
